<a href="https://www.kaggle.com/code/sundarmiral/resnet50-cifar10?scriptVersionId=92904667" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import tensorflow as tf

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.applications import ResNet50
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tensorflow.keras import optimizers
#from tensorflow.keras.callbacks import TensorBoard
#import Talos as ta
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

cifar = tf.keras.datasets.cifar10
(x_train, y_train),(x_test, y_test) = cifar.load_data()

X = np.concatenate((x_train,x_test))
y = np.concatenate((y_train,y_test))

#to shuffle the images
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=34)

#train data = 70 %
#test data =15%
#Validation data =15 %



x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

#y_train = keras.utils.to_categorical(y_train, 10)
#y_test = keras.utils.to_categorical(y_test, 10)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

#splitting into Validation and Training
X_train, x_val, Y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=33)



batch_size = 20  # The default batch size of keras.
num_classes = 10  # Number of class for the dataset
epochs = 12
data_augmentation = False

datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)
datagen.fit(X_train)

datagenFeaturewiseCenter = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=True,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=False,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)
datagenFeaturewiseCenter.fit(X_train)

datagenrotate = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=0,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=45,
        # randomly shift images horizontally
        width_shift_range=0,
        # randomly shift images vertically
        height_shift_range=0,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=False,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)
datagenrotate.fit(X_train)

# dense_layers = [1]
# layer_sizes = [64]
# conv_layers = [2]

lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.1, patience=3, verbose=1,
            mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.000001
        )
es = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', min_delta=0, patience=5, verbose=1,
            mode='auto', baseline=None, restore_best_weights=True
        )

# x_train =  np.reshape(x_train, (224,224,3))
# y_train =  np.reshape(y_train, (224,224,3))

# x_test =  np.reshape(x_test, (224,224,3))
# y_test =  np.reshape(y_test, (224,224,3))

# x_train.reshape(x_train.size,224,224,3)
# y_train.reshape(y_train.size,224,224,3)

# x_test.reshape(x_test.size,224,224,3)
# y_test.reshape(y_test.size,224,224,3)

# for image in x_train:
#     image.reshape(image.shape[0],224,224,3)
# for image1 in y_train:
#     image1.reshape(image.shape[0],224,224,3)
# for image2 in x_test:
#     image2.reshape(image.shape[0],224,224,3)
# for image3 in y_test:
#     image3.reshape(image.shape[0],224,224,3)


# x_train = x_train.reshape(x_train.shape[0],224,224,3)
# y_train = y_train.reshape(y_train.shape[0],224,224,3)
# x_test = x_test.reshape(x_test.shape[0],224,224,3)
# y_test = y_test.reshape(y_test.shape[0],224,224,3)


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.UpSampling2D((2,2)))
model.add(tf.keras.layers.UpSampling2D((2,2)))
model.add(tf.keras.layers.UpSampling2D((2,2)))

resnet = ResNet50(
    include_top=False, weights='imagenet',
    input_shape=(256,256,3)
)


model.add(resnet)

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(10, activation='softmax'))
#model.add(tf.keras.layers.Dense(128, activation='relu'))
#model.add(tf.keras.layers.Dropout(0.5))
#model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer=optimizers.RMSprop(lr=2e-5),
  metrics=['accuracy'])
#2e-5


history  = model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,validation_data =(x_val,y_val),
              shuffle=True, callbacks=[lr])

#with Data Augmentation Horizontal flip
history = model.fit(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    validation_data =datagen.flow(x_val, y_val, batch_size=5), epochs=epochs, callbacks=[lr,es], shuffle=True)

#with Data Augmentation featurewise center
history = model.fit(datagenFeaturewiseCenter.flow(X_train, Y_train, batch_size=batch_size),
                    validation_data =datagenFeaturewiseCenter.flow(x_val, y_val, batch_size=5), epochs=epochs, callbacks=[lr,es], shuffle=True)



# #with Data Augmentation Rotate
# history = model.fit(datagenrotate.flow(X_train, Y_train, batch_size=batch_size),
#                     validation_data =datagenrotate.flow(x_val, y_val, batch_size=5), epochs=epochs, callbacks=[lr], shuffle=True)

#validation_split=0.15,
model.save('/kaggle/working/models/resnet_dataAug_17-05.h5')

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()



#Confusion matrix plot
cmap=plt.cm.Blues
p_test = model.predict(x_test).argmax(axis=1)
#print(p_test[0:20],y_test[0:20])
#print(p_test[0:20],y_test[0:20])
#p_test_mod = p_test.ravel()
newy = [np.where(r==1)[0][0] for r in y_test]
cm = confusion_matrix(newy, p_test)
print("Confusion Matrix:",cm)

plt.imshow(cm, interpolation='nearest', cmap=cmap)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, list(range(10)), rotation=45)
plt.yticks(tick_marks, list(range(10)))

fmt ='d'
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
  plt.text(j, i, format(cm[i, j], fmt),
           horizontalalignment="center",
           color="white" if cm[i, j] > thresh else "black")
    
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=32)
print("test loss, test acc:", results)
print("\n")
model.summary()


In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


In [ ]:
#print(y_val.size,x_val.size,p_test.size)

p_test = model.predict(x_test).argmax(axis=1)
#print(p_test[0:20],y_test[0:20])
newy = [np.where(r==1)[0][0] for r in y_test]
#print(newy)
#p_test_mod = p_test.ravel()
#y_test_ravel = y_test.ravel()
cm = confusion_matrix(newy, p_test)
print("Confusion Matrix:",cm)

plt.imshow(cm, interpolation='nearest', cmap=cmap)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, list(range(10)), rotation=45)
plt.yticks(tick_marks, list(range(10)))
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()